In [2]:
import numpy as np
import pandas as pd
#https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction
#https://www.kaggle.com/datasets/alexisbcook/synthetic-credit-card-approval
import os
import kagglehub


df = pd.read_csv("/opt/notebooks/Chatbot-Credit-Card/backend//dataset/Cleaned-CC-Data/clean_dataset.csv")
df

# path = kagglehub.dataset_download("rikdifos/credit-card-approval-prediction")

# print("Path to dataset files:", path)

# current_working_dir = os.getcwd()

# os.system(f"mv {path} {destination_path}")

# csv_files = [f for f in os.listdir(destination_path) if f.endswith('.csv')]
# dataframes = {}
# print(f"Dataset moved to: {destination_path}")

# csv_files = [f for f in os.listdir(destination_path) if f.endswith('.csv')]

# # Dynamically load each CSV into its own DataFrame using globals()
# for csv_file in csv_files:
#     file_path = os.path.join(destination_path, csv_file)
#     df_name = "df_" + os.path.splitext(csv_file)[0]  # Create a name for the DataFrame (e.g., df_credit_record)
    
#     # Dynamically assign the DataFrame to a variable using globals()
#     globals()[df_name] = pd.read_csv(file_path)
    
#     print(f"Loaded {csv_file} into DataFrame: {df_name}")

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.085,0,0,Education,Black,1.25,0,0,0,0,ByBirth,260,0,0
686,0,22.67,0.750,1,1,Energy,White,2.00,0,1,2,1,ByBirth,200,394,0
687,0,25.25,13.500,0,0,Healthcare,Latino,2.00,0,1,1,1,ByBirth,200,1,0
688,1,17.92,0.205,1,1,ConsumerStaples,White,0.04,0,0,0,0,ByBirth,280,750,0


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import colorama
from colorama import Fore

# Automatically identify categorical columns from the dataframe based on data types
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Identify potential categorical columns stored as numbers by checking unique values
for col in df.columns:
    if df[col].nunique() < 10 and col not in categorical_cols:  # Arbitrary threshold of < 10 unique values
        categorical_cols.append(col)

# Display identified categorical columns
print(f"Identified Categorical Columns: {categorical_cols}\n")


Identified Categorical Columns: ['Industry', 'Ethnicity', 'Citizen', 'Gender', 'Married', 'BankCustomer', 'PriorDefault', 'Employed', 'DriversLicense', 'Approved']



In [4]:
# Initialize a list to store bad predictors
bad_predictors = []

# You can change the threshold for what is considered a "bad predictor"
p_value_threshold = 0.05  # Adjust this if needed

# Evaluate each categorical column using chi-squared test
for col in categorical_cols:
    print(f"{col}:")
    
    # Perform chi-squared test
    a = np.array(pd.crosstab(df['Approved'], df[col]))
    stats, p, dof, _ = chi2_contingency(a, correction=False)
    
    if p > p_value_threshold:
        print(Fore.RED + f"'{col}' is a 'bad Predictor'")
        print(f"p_val = {p}\n")
        bad_predictors.append(col)  # Append the bad predictor to the list
    else:
        print(Fore.GREEN + f"'{col}' is a 'Good Predictor'")
        print(f"p_val = {p}\n")

Industry:
'Industry' is a 'Good Predictor'
p_val = 3.502987066102042e-15

Ethnicity:
'Ethnicity' is a 'Good Predictor'
p_val = 1.823665654934685e-08

Citizen:
'Citizen' is a 'Good Predictor'
p_val = 0.010094291370456362

Gender:
'Gender' is a 'bad Predictor'
p_val = 0.44723087514133186

Married:
'Married' is a 'Good Predictor'
p_val = 2.100231920165588e-06

BankCustomer:
'BankCustomer' is a 'Good Predictor'
p_val = 6.91661320541803e-07

PriorDefault:
'PriorDefault' is a 'Good Predictor'
p_val = 7.298530125411298e-80

Employed:
'Employed' is a 'Good Predictor'
p_val = 2.227269345312281e-33

DriversLicense:
'DriversLicense' is a 'bad Predictor'
p_val = 0.4061341323141693

Approved:
'Approved' is a 'Good Predictor'
p_val = 4.469841378183071e-152



In [5]:
# Automatically drop bad predictors
df.drop(bad_predictors, axis=1, inplace=True)

# Print the final dataframe structure after dropping
print(f"Dropped bad predictors: {bad_predictors}")
print(df.head())

Dropped bad predictors: ['Gender', 'DriversLicense']
     Age   Debt  Married  BankCustomer     Industry Ethnicity  YearsEmployed  \
0  30.83  0.000        1             1  Industrials     White           1.25   
1  58.67  4.460        1             1    Materials     Black           3.04   
2  24.50  0.500        1             1    Materials     Black           1.50   
3  27.83  1.540        1             1  Industrials     White           3.75   
4  20.17  5.625        1             1  Industrials     White           1.71   

   PriorDefault  Employed  CreditScore       Citizen  ZipCode  Income  \
0             1         1            1       ByBirth      202       0   
1             1         1            6       ByBirth       43     560   
2             1         0            0       ByBirth      280     824   
3             1         1            5       ByBirth      100       3   
4             1         0            0  ByOtherMeans      120       0   

   Approved  
0         1  

In [6]:
# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove the target variable from the list of features if present
if 'Approved' in categorical_cols:
    categorical_cols.remove('Approved')
if 'Approved' in numerical_cols:
    numerical_cols.remove('Approved')


In [7]:
print("Data types of the columns:")
print(df.dtypes)

print("\nCategorical columns identified:")
print(categorical_cols)


Data types of the columns:
Age              float64
Debt             float64
Married            int64
BankCustomer       int64
Industry          object
Ethnicity         object
YearsEmployed    float64
PriorDefault       int64
Employed           int64
CreditScore        int64
Citizen           object
ZipCode            int64
Income             int64
Approved           int64
dtype: object

Categorical columns identified:
['Industry', 'Ethnicity', 'Citizen']


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Initialize LabelEncoders for categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store the encoder for future use (e.g., inverse_transform)

print("Unique values in categorical columns after encoding:")
for col in categorical_cols:
    print(f"{col}: {df[col].unique()}")
print("\nData types of the columns after encoding:")
print(df.dtypes)

Unique values in categorical columns after encoding:
Industry: [ 7  9  0 12  8  5  4 10 13  1  3  2  6 11]
Ethnicity: [4 1 0 2 3]
Citizen: [0 1 2]

Data types of the columns after encoding:
Age              float64
Debt             float64
Married            int64
BankCustomer       int64
Industry           int64
Ethnicity          int64
YearsEmployed    float64
PriorDefault       int64
Employed           int64
CreditScore        int64
Citizen            int64
ZipCode            int64
Income             int64
Approved           int64
dtype: object


In [10]:
# For preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# For splitting the data
from sklearn.model_selection import train_test_split

# Scale numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Separate features and target variable
X = df.drop('Approved', axis=1)
y = df['Approved']

# Split the data into training, testing, and validation sets
# First split: train + validate and test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Second split: train and validate
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Print the shapes of the resulting splits
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set: (414, 13) (414,)
Validation set: (138, 13) (138,)
Test set: (138, 13) (138,)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, roc_auc_score

# List of models to evaluate
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42, ccp_alpha=0.01),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "XGBoost": XGBClassifier(random_state=42, eval_metric='logloss')
}

# Dictionary to store evaluation results
results = {}

# Iterate over each model
for model_name, model in models.items():
    # Train the model using the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test and validation data
    y_test_pred = model.predict(X_test)
    y_val_pred = model.predict(X_val)
    
    # For AUC-ROC, we need the probability scores for the positive class
    y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    y_val_prob = model.predict_proba(X_val)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Evaluate the model on the test set
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_auc = roc_auc_score(y_test, y_test_prob) if y_test_prob is not None else None
    
    # Evaluate the model on the validation set
    val_mae = mean_absolute_error(y_val, y_val_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_auc = roc_auc_score(y_val, y_val_prob) if y_val_prob is not None else None
    
    # Store the evaluation results
    results[model_name] = {
        "Test Accuracy": test_accuracy,
        "Test AUC": test_auc,
        "Validation Accuracy": val_accuracy,
        "Validation AUC": val_auc,
        "Test MAE": test_mae,
        "Validation MAE": val_mae,
        "Test R2": test_r2,
        "Validation R2": val_r2
    }

# Display the raw numbers
print("\nModel Evaluation Results:")
for model_name, metrics in results.items():
    print(f"\n{model_name} Evaluation:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

In [ ]:
# Find the best model based on the highest test accuracy
best_model = max(results, key=lambda x: results[x]["Test Accuracy"])
best_accuracy = results[best_model]["Test Accuracy"]

print(f"\nBest Model: {best_model}")
print(f"Best Test Accuracy: {best_accuracy:.4f}")

# Access the trained model object
best_model_name = best_model  # 'Random Forest' as per your previous code
best_model = models[best_model_name]  # This is the trained Random Forest model



In [ ]:
from lime.lime_tabular import LimeTabularExplainer
import numpy as np
import pandas as pd
import warnings

# Suppress specific warnings from LIME (optional)
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the LimeTabularExplainer
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['Not Approved', 'Approved'],
    mode='classification'
)
# Initialize an empty list to store explanations
explanations = []


# Suppress the specific warnings from the deprecated usage in lime
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the LimeTabularExplainer
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['Not Approved', 'Approved'],  # Adjust according to your dataset
    mode='classification'
)

# Pick a single instance from the test set to explain for testing
i = 0  # Index of the instance to explain

exp = lime_explainer.explain_instance(X_test.iloc[i], best_model.predict_proba)

# Display the explanation for the instance
exp.show_in_notebook(show_table=True)


In [ ]:
import re

# Suppress specific warnings from LIME (optional)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to extract the feature name from the LIME explanation
def extract_feature_name(feature_str):
    # Split on '<', '>', '=', '!=', etc., and take the first part
    split_symbols = ['<=', '>=', '<', '>', '!=', '=']
    for symbol in split_symbols:
        if symbol in feature_str:
            feature_name = feature_str.split(symbol)[0].strip()
            break
    else:
        feature_name = feature_str.strip()
    # Remove any numeric values
    feature_name = re.sub(r'[0-9]+', '', feature_name)
    # Remove any non-alphanumeric characters except spaces
    feature_name = re.sub(r'[^A-Za-z0-9 ]+', '', feature_name)
    # Replace multiple spaces with a single space
    feature_name = re.sub(r'\s+', ' ', feature_name)
    return feature_name.strip()

# Initialize the LimeTabularExplainer for the entire feature dataset `X`
lime_explainer = LimeTabularExplainer(
    training_data=np.array(X),
    feature_names=X.columns,
    class_names=['Not Approved', 'Approved'],  # Adjust according to your dataset
    mode='classification'
)

# Initialize an empty list to store explanations
explanations = []

# Loop over the instances in the entire feature dataset `X`
for idx, instance in X.iterrows():
    # Get the model's prediction
    prediction = best_model.predict(instance.values.reshape(1, -1))[0]
    # Get the actual label from `y`
    actual_label = y.loc[idx]
    # Generate LIME explanation
    exp = lime_explainer.explain_instance(
        data_row=instance,
        predict_fn=best_model.predict_proba,
        num_features=len(X.columns)  # Consider all features
    )
    # Get the list of (feature, contribution) pairs
    explanation_list = exp.as_list()

    # Determine the decision (Approved or Denied)
    decision = 'Approved' if prediction == 1 else 'Denied'

    # Collect features contributing to the predicted class
    features_contributing = []
    for feature, weight in explanation_list:
        # For the predicted class, collect all features regardless of weight
        if (prediction == 1 and weight > 0) or (prediction == 0 and weight < 0):
            feature_name = extract_feature_name(feature)
            if feature_name and feature_name not in features_contributing:
                features_contributing.append(feature_name)

    # Construct the explanation text
    if features_contributing:
        explanation_text = f"This application was {decision.lower()} due to " + ", ".join(features_contributing) + "."
    else:
        explanation_text = f"This application was {decision.lower()}."

    # Append the explanation and related information
    explanations.append({
        'Index': idx,
        'Prediction': decision,
        'Actual': 'Approved' if actual_label == 1 else 'Denied',
        'Explanation': explanation_text
    })


In [ ]:
# Convert the explanations to a DataFrame
explanations_df = pd.DataFrame(explanations)

# Output the first explanation
first_explanation = explanations_df.iloc[0]

# Print the details
print("Index:", first_explanation['Index'])
print("Prediction:", first_explanation['Prediction'])
print("Actual:", first_explanation['Actual'])
print("Explanation:", first_explanation['Explanation'])


In [ ]:
explanations_df

In [ ]:
#reimport data due to earlier augmentation
df = pd.read_csv("../dataset/Cleaned-CC-Data/clean_dataset.csv")
df

In [ ]:
# Add the 'Explanation' column (Reason) from explanations_df to the original df
df['Reason'] = explanations_df['Explanation']

# Save the updated dataframe with the new column as a CSV file
df.to_csv('../dataset/cleaned-augmented.csv', index=False)

In [ ]:
df